In [27]:
# Install the following packages incase they're not loaded already
!pip install pdfplumber
!pip install chromadb
!pip install tiktoken
!pip install openai


In [28]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
import openai

# Assuming 'filepath' is defined and points to the directory of your OpenAI_API_Key.txt file
with open(filepath + "OpenAI_API_Key.txt", "r") as f:
    openai.api_key = ' '.join(f.readlines()).strip()


In [30]:
## Issues with normal LLMs
messages = [
    {"role":"system", "content":"You are an AI assistant to user."},
    {"role":"user", "content":"Can you explain the process for filing a claim under this life insurance policy?"},
          ]

In [31]:
## New method (as older completion and chatcompletion endpoints have been deprecated)
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages)
response.choices[0].message.content

"Sure! Here are the general steps you may need to follow to file a claim under a life insurance policy:\n\n1. **Notify the Insurance Company**: Contact the insurance company as soon as possible after the policyholder passes away. You can usually find the contact information on the policy documents or the company's website.\n\n2. **Gather Documents**: The insurance company will require certain documents to process the claim. This typically includes the death certificate, a claim form (provided by the insurance company), and any other documents they specify.\n\n3. **Complete Claim Form**: Fill out the claim form accurately and provide all the requested information. Make sure to include the policy number and the details of the policyholder's death.\n\n4. **Submit Documents**: Send the completed claim form and all required documents to the insurance company by mail or through an online portal, depending on their preferred method.\n\n5. **Wait for Review**: The insurance company will review

In [32]:
pdf_path = "/content/drive/MyDrive/HelpMate/"

In [33]:
# Function to check whether a word is present in a table or not for segregation of regular text and tables

def check_bboxes(word, table_bbox):
    # Check whether word is inside a table bbox.
    l = word['x0'], word['top'], word['x1'], word['bottom']
    r = table_bbox
    return l[0] > r[0] and l[1] > r[1] and l[2] < r[2] and l[3] < r[3]

In [34]:
# Function to extract text from a PDF file.
# 1. Declare a variable p to store the iteration of the loop that will help us store page numbers alongside the text
# 2. Declare an empty list 'full_text' to store all the text files
# 3. Use pdfplumber to open the pdf pages one by one
# 4. Find the tables and their locations in the page
# 5. Extract the text from the tables in the variable 'tables'
# 6. Extract the regular words by calling the function check_bboxes() and checking whether words are present in the table or not
# 7. Use the cluster_objects utility to cluster non-table and table words together so that they retain the same chronology as in the original PDF
# 8. Declare an empty list 'lines' to store the page text
# 9. If a text element in present in the cluster, append it to 'lines', else if a table element is present, append the table
# 10. Append the page number and all lines to full_text, and increment 'p'
# 11. When the function has iterated over all pages, return the 'full_text' list

def extract_text_from_pdf(pdf_path):
    p = 0
    full_text = []


    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_no = f"Page {p+1}"
            text = page.extract_text()

            tables = page.find_tables()
            table_bboxes = [i.bbox for i in tables]
            tables = [{'table': i.extract(), 'top': i.bbox[1]} for i in tables]
            non_table_words = [word for word in page.extract_words() if not any(
                [check_bboxes(word, table_bbox) for table_bbox in table_bboxes])]
            lines = []

            for cluster in pdfplumber.utils.cluster_objects(non_table_words + tables, itemgetter('top'), tolerance=5):

                if 'text' in cluster[0]:
                    try:
                        lines.append(' '.join([i['text'] for i in cluster]))
                    except KeyError:
                        pass

                elif 'table' in cluster[0]:
                    lines.append(json.dumps(cluster[0]['table']))


            full_text.append([page_no, " ".join(lines)])
            p +=1

    return full_text

In [37]:
from pathlib import Path
import pandas as pd
import pdfplumber

# Define the directory containing the PDF files
pdf_directory = Path("/content/drive/MyDrive/HelpMate/")  # Update this path

# Initialize an empty list to store the extracted texts and document names
data = []

def extract_text_from_pdf(pdf_path):
    text_by_page = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:  # Check if text extraction was successful
                text_by_page.append(text)
    return text_by_page

# Loop through all files in the directory
for pdf_path in pdf_directory.glob("*.pdf"):
    print(f"...Processing {pdf_path.name}")
    extracted_text = extract_text_from_pdf(pdf_path)

    # Assuming you want to keep each page's text separate for now
    extracted_text_df = pd.DataFrame(extracted_text, columns=['Page_Text'])
    extracted_text_df['Page No.'] = extracted_text_df.index + 1
    extracted_text_df['Document Name'] = pdf_path.name

    data.append(extracted_text_df)
    print(f"Finished processing {pdf_path.name}")

# Concatenate all dataframes into a single one
all_data_df = pd.concat(data, ignore_index=True)
print("All PDFs have been processed.")

...Processing Principal-Sample-Life-Insurance-Policy.pdf
Finished processing Principal-Sample-Life-Insurance-Policy.pdf
All PDFs have been processed.


In [38]:
Insurance_pdfs_data = pd.concat(data, ignore_index=True)

In [39]:
Insurance_pdfs_data.head(5)

,Page_Text,Page No.,Document Name
0,DOROTHEA GLAUSE S655\nRHODE ISLAND JOHN DOE 01...,1,Principal-Sample-Life-Insurance-Policy.pdf
1,This page left blank intentionally,2,Principal-Sample-Life-Insurance-Policy.pdf
2,POLICY RIDER\nGROUP INSURANCE\nPOLICY NO: S655...,3,Principal-Sample-Life-Insurance-Policy.pdf
3,This page left blank intentionally,4,Principal-Sample-Life-Insurance-Policy.pdf
4,PRINCIPAL LIFE INSURANCE COMPANY\n(called The ...,5,Principal-Sample-Life-Insurance-Policy.pdf


In [42]:
len(Insurance_pdfs_data)

64

In [43]:
Insurance_pdfs_data['Metadata'] = Insurance_pdfs_data.apply(lambda x: {'filing_name': x['Document Name'][:-4], 'Page_No.': x['Page No.']}, axis=1)

In [44]:
Insurance_pdfs_data.head(5)

,Page_Text,Page No.,Document Name,Metadata
0,DOROTHEA GLAUSE S655\nRHODE ISLAND JOHN DOE 01...,1,Principal-Sample-Life-Insurance-Policy.pdf,{'filing_name': 'Principal-Sample-Life-Insuran...
1,This page left blank intentionally,2,Principal-Sample-Life-Insurance-Policy.pdf,{'filing_name': 'Principal-Sample-Life-Insuran...
2,POLICY RIDER\nGROUP INSURANCE\nPOLICY NO: S655...,3,Principal-Sample-Life-Insurance-Policy.pdf,{'filing_name': 'Principal-Sample-Life-Insuran...
3,This page left blank intentionally,4,Principal-Sample-Life-Insurance-Policy.pdf,{'filing_name': 'Principal-Sample-Life-Insuran...
4,PRINCIPAL LIFE INSURANCE COMPANY\n(called The ...,5,Principal-Sample-Life-Insurance-Policy.pdf,{'filing_name': 'Principal-Sample-Life-Insuran...


In [45]:
# Import the OpenAI Embedding Function into chroma

from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

In [51]:
# Define the path where chroma collections will be stored

chroma_data_path = "/content/drive/MyDrive/HelpMate/ChromaDB_Data"

In [54]:
import chromadb

# Call PersistentClient()
client = chromadb.PersistentClient(path=chroma_data_path)

In [ ]:
import chromadb

# Call PersistentClient()
client = chromadb.PersistentClient(path=chroma_data_path)

In [55]:
# Set up the embedding function using the OpenAI embedding model

model = "text-embedding-ada-002"
embedding_function = OpenAIEmbeddingFunction(api_key=openai.api_key, model_name=model)

In [64]:
## Create an empty collection
Insurance_collection = client.get_or_create_collection(name='RAG_on_Uber', embedding_function=embedding_function)

In [65]:
documents_list = Insurance_pdfs_data["Page_Text"].tolist()
metadata_list = Insurance_pdfs_data['Metadata'].tolist()

In [66]:
# Add the documents and metadata to the collection alongwith generic integer IDs. You can also feed the metadata information as IDs by combining the policy name and page no.

Insurance_collection.add(
    documents= documents_list,
    ids = [str(i) for i in range(0, len(documents_list))],
    metadatas = metadata_list
)

In [67]:
Insurance_collection.peek(1)

{'ids': ['0'],
 'embeddings': [[-0.022534452378749847,
   0.018737390637397766,
   -0.02727099508047104,
   -0.03126377984881401,
   -0.003699199529364705,
   0.02787121757864952,
   0.0008636358543299139,
   0.02363051287829876,
   -0.01663661189377308,
   0.00434509152546525,
   -0.005235639400780201,
   0.004116745665669441,
   -0.006269718986004591,
   0.023304304108023643,
   -0.01122808177024126,
   0.031055007129907608,
   0.0028543206863105297,
   -0.024648280814290047,
   0.03071575239300728,
   0.007848566398024559,
   0.00643608532845974,
   0.018933115527033806,
   0.00962313823401928,
   0.01257858332246542,
   -0.0124807208776474,
   -0.009473082609474659,
   0.017889250069856644,
   -0.021673263981938362,
   0.02357831969857216,
   0.003056569490581751,
   0.01328971702605486,
   -0.014614121057093143,
   -0.03358638286590576,
   -0.041128311306238174,
   -0.008461837656795979,
   0.01368769071996212,
   -0.002546053845435381,
   -0.0039242831990122795,
   0.012369809672

In [68]:
cache_collection = client.get_or_create_collection(name='Insurance_Cache', embedding_function=embedding_function)

In [69]:
cache_collection.peek()

{'ids': [],
 'embeddings': [],
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None}

In [70]:
# Read the user query
query = input()

Effective Date for Benefit Changes Due to Change by Policy Amendment


In [72]:
## Let's query the above result
?Insurance_collection.query

In [74]:
## Quickly checking the results of the query
results = Insurance_collection.query(
      query_texts=query,
      n_results=10
      )

In [75]:
results

{'ids': [['30', '29', '27', '20', '31', '15', '28', '23', '42', '26']],
 'distances': [[0.23270089336899377,
   0.23651484913040724,
   0.2973989921201877,
   0.3185787060626363,
   0.3205899405885721,
   0.32493935307687477,
   0.3293245390118688,
   0.3324978230134449,
   0.34103955069482433,
   0.34292997382838036]],
 'metadatas': [[{'Page_No.': 31,
    'filing_name': 'Principal-Sample-Life-Insurance-Policy'},
   {'Page_No.': 30, 'filing_name': 'Principal-Sample-Life-Insurance-Policy'},
   {'Page_No.': 28, 'filing_name': 'Principal-Sample-Life-Insurance-Policy'},
   {'Page_No.': 21, 'filing_name': 'Principal-Sample-Life-Insurance-Policy'},
   {'Page_No.': 32, 'filing_name': 'Principal-Sample-Life-Insurance-Policy'},
   {'Page_No.': 16, 'filing_name': 'Principal-Sample-Life-Insurance-Policy'},
   {'Page_No.': 29, 'filing_name': 'Principal-Sample-Life-Insurance-Policy'},
   {'Page_No.': 24, 'filing_name': 'Principal-Sample-Life-Insurance-Policy'},
   {'Page_No.': 43, 'filing_name': 'P

In [76]:
for key, val in results.items():
  print(key)

ids
distances
metadatas
embeddings
documents
uris
data


In [77]:
for key, val in results.items():
  print(val)

[['30', '29', '27', '20', '31', '15', '28', '23', '42', '26']]
[[0.23270089336899377, 0.23651484913040724, 0.2973989921201877, 0.3185787060626363, 0.3205899405885721, 0.32493935307687477, 0.3293245390118688, 0.3324978230134449, 0.34103955069482433, 0.34292997382838036]]
[[{'Page_No.': 31, 'filing_name': 'Principal-Sample-Life-Insurance-Policy'}, {'Page_No.': 30, 'filing_name': 'Principal-Sample-Life-Insurance-Policy'}, {'Page_No.': 28, 'filing_name': 'Principal-Sample-Life-Insurance-Policy'}, {'Page_No.': 21, 'filing_name': 'Principal-Sample-Life-Insurance-Policy'}, {'Page_No.': 32, 'filing_name': 'Principal-Sample-Life-Insurance-Policy'}, {'Page_No.': 16, 'filing_name': 'Principal-Sample-Life-Insurance-Policy'}, {'Page_No.': 29, 'filing_name': 'Principal-Sample-Life-Insurance-Policy'}, {'Page_No.': 24, 'filing_name': 'Principal-Sample-Life-Insurance-Policy'}, {'Page_No.': 43, 'filing_name': 'Principal-Sample-Life-Insurance-Policy'}, {'Page_No.': 27, 'filing_name': 'Principal-Sample-Li

In [78]:
# Searh the Cache collection first
# Query the collection against the user query and return the top result

cache_results = cache_collection.query(
    query_texts=query,
    n_results=1
)

In [79]:
# Implementing Cache in Semantic Search

# Set a threshold for cache search
threshold = 0.2

ids = []
documents = []
distances = []
metadatas = []
results_df = pd.DataFrame()


# If the distance is greater than the threshold, then return the results from the main collection.

if cache_results['distances'][0] == [] or cache_results['distances'][0][0] > threshold:
      # Query the collection against the user query and return the top 10 results
      results = financedata_collection.query(
      query_texts=query,
      n_results=10
      )

      # Store the query in cache_collection as document w.r.t to ChromaDB so that it can be embedded and searched against later
      # Store retrieved text, ids, distances and metadatas in cache_collection as metadatas, so that they can be fetched easily if a query indeed matches to a query in cache
      Keys = []
      Values = []

      for key, val in results.items():
        if key not in ['embeddings', 'uris','data']:
          for i in range(10):
            Keys.append(str(key)+str(i))
            Values.append(str(val[0][i]))


      cache_collection.add(
          documents= [query],
          ids = [query],  # Or if you want to assign integers as IDs 0,1,2,.., then you can use "len(cache_results['documents'])" as will return the no. of queries currently in the cache and assign the next digit to the new query."
          metadatas = dict(zip(Keys, Values))
      )

      print("Not found in cache. Found in main collection.")

      result_dict = {'Metadatas': results['metadatas'][0], 'Documents': results['documents'][0], 'Distances': results['distances'][0], "IDs":results["ids"][0]}
      results_df = pd.DataFrame.from_dict(result_dict)
      results_df


# If the distance is, however, less than the threshold, you can return the results from cache

elif cache_results['distances'][0][0] <= threshold:
      cache_result_dict = cache_results['metadatas'][0][0]

      # Loop through each inner list and then through the dictionary
      for key, value in cache_result_dict.items():
          if 'ids' in key:
              ids.append(value)
          elif 'documents' in key:
              documents.append(value)
          elif 'distances' in key:
              distances.append(value)
          elif 'metadatas' in key:
              metadatas.append(value)

      print("Found in cache!")

      # Create a DataFrame
      results_df = pd.DataFrame({
        'IDs': ids,
        'Documents': documents,
        'Distances': distances,
        'Metadatas': metadatas
      })

Not found in cache. Found in main collection.


In [80]:
results_df

,Metadatas,Documents,Distances,IDs
0,"{'Page_No.': 31, 'filing_name': 'Principal-Sam...",Scheduled Benefit in force for the Member befo...,0.232701,30
1,"{'Page_No.': 30, 'filing_name': 'Principal-Sam...","(6) If, on the date a Member becomes eligible ...",0.236515,29
2,"{'Page_No.': 28, 'filing_name': 'Principal-Sam...",Section B - Effective Dates\nArticle 1 - Membe...,0.297399,27
3,"{'Page_No.': 21, 'filing_name': 'Principal-Sam...",b . on any date the definition of Member or De...,0.318579,20
4,"{'Page_No.': 32, 'filing_name': 'Principal-Sam...",(1) marriage or establishment of a Civil Union...,0.320590,31
5,"{'Page_No.': 16, 'filing_name': 'Principal-Sam...",PART II - POLICY ADMINISTRATION\nSection A - C...,0.324939,15
6,"{'Page_No.': 29, 'filing_name': 'Principal-Sam...",Insurance for which Proof of Good Health is re...,0.329325,28
7,"{'Page_No.': 24, 'filing_name': 'Principal-Sam...",T he Principal may terminate the Policyholder'...,0.332498,23
8,"{'Page_No.': 43, 'filing_name': 'Principal-Sam...",Any individual policy issued will then be in f...,0.341040,42
9,"{'Page_No.': 27, 'filing_name': 'Principal-Sam...",I f a Member's Dependent is employed and is co...,0.342930,26


In [81]:
## Checking if the cache also contains the results
cache_results = cache_collection.query(
    query_texts=query,
    n_results=1
)

In [82]:
!pip install sentence_transformers

In [83]:
# Import the CrossEncoder library from sentence_transformers

from sentence_transformers import CrossEncoder, util

In [84]:
# Initialise the cross encoder model

cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [86]:
scores = cross_encoder.predict([['What services does The Principal offer to employer groups besides life insurance coverage',
 'How does The Principal handle the provision and liability of third-party services and discounts'],
                                ['What are the primary benefits and provisions outlined in the group policy for Rhode Island John Doe', 'Describe the eligibility criteria for member life insurance under this policy']])

In [87]:
scores

array([ -2.4576452, -10.936317 ], dtype=float32)

In [88]:
# Input (query, response) pairs for each of the top 20 responses received from the semantic search to the cross encoder
# Generate the cross_encoder scores for these pairs

cross_inputs = [[query, response] for response in results_df['Documents']]
cross_rerank_scores = cross_encoder.predict(cross_inputs)

In [89]:
 #Store the rerank_scores in results_df

results_df['Reranked_scores'] = cross_rerank_scores

In [90]:
results_df

,Metadatas,Documents,Distances,IDs,Reranked_scores
0,"{'Page_No.': 31, 'filing_name': 'Principal-Sam...",Scheduled Benefit in force for the Member befo...,0.232701,30,6.856415
1,"{'Page_No.': 30, 'filing_name': 'Principal-Sam...","(6) If, on the date a Member becomes eligible ...",0.236515,29,5.220098
2,"{'Page_No.': 28, 'filing_name': 'Principal-Sam...",Section B - Effective Dates\nArticle 1 - Membe...,0.297399,27,1.359932
3,"{'Page_No.': 21, 'filing_name': 'Principal-Sam...",b . on any date the definition of Member or De...,0.318579,20,1.308147
4,"{'Page_No.': 32, 'filing_name': 'Principal-Sam...",(1) marriage or establishment of a Civil Union...,0.320590,31,0.830044
5,"{'Page_No.': 16, 'filing_name': 'Principal-Sam...",PART II - POLICY ADMINISTRATION\nSection A - C...,0.324939,15,2.617916
6,"{'Page_No.': 29, 'filing_name': 'Principal-Sam...",Insurance for which Proof of Good Health is re...,0.329325,28,-2.554943
7,"{'Page_No.': 24, 'filing_name': 'Principal-Sam...",T he Principal may terminate the Policyholder'...,0.332498,23,-2.261049
8,"{'Page_No.': 43, 'filing_name': 'Principal-Sam...",Any individual policy issued will then be in f...,0.341040,42,-2.264925
9,"{'Page_No.': 27, 'filing_name': 'Principal-Sam...",I f a Member's Dependent is employed and is co...,0.342930,26,-6.790921


In [91]:
# Return the top 3 results from semantic search
top_3_semantic = results_df.sort_values(by='Distances')
top_3_semantic[:3]

,Metadatas,Documents,Distances,IDs,Reranked_scores
0,"{'Page_No.': 31, 'filing_name': 'Principal-Sam...",Scheduled Benefit in force for the Member befo...,0.232701,30,6.856415
1,"{'Page_No.': 30, 'filing_name': 'Principal-Sam...","(6) If, on the date a Member becomes eligible ...",0.236515,29,5.220098
2,"{'Page_No.': 28, 'filing_name': 'Principal-Sam...",Section B - Effective Dates\nArticle 1 - Membe...,0.297399,27,1.359932


In [92]:
# Return the top 3 results after reranking

top_3_rerank = results_df.sort_values(by='Reranked_scores', ascending=False)
top_3_rerank[:3]
top_3_RAG = top_3_rerank[["Documents", "Metadatas"]][:3]

In [93]:
top_3_RAG

,Documents,Metadatas
0,Scheduled Benefit in force for the Member befo...,"{'Page_No.': 31, 'filing_name': 'Principal-Sam..."
1,"(6) If, on the date a Member becomes eligible ...","{'Page_No.': 30, 'filing_name': 'Principal-Sam..."
5,PART II - POLICY ADMINISTRATION\nSection A - C...,"{'Page_No.': 16, 'filing_name': 'Principal-Sam..."


In [94]:
retrieved = top_3_RAG['Documents'][0]

In [95]:
messages = [
    {"role":"system", "content":"You are an AI assistant to user."},
    {"role":"user", "content":f"""How does the policy define "Actively at Work" and its implications for insurance eligibility?' """},
          ]

In [96]:
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages)
response.choices[0].message.content

'"Actively at Work" typically refers to an employee\'s physical presence at their place of employment during scheduled working hours. Insurance policies, such as health or life insurance, often require employees to be actively at work in order to be eligible for coverage. If an employee is not actively at work due to various reasons such as illness, injury, or leave of absence, they may not be eligible for insurance coverage during that period. It\'s important to review the specific terms and conditions of the insurance policy to understand how "Actively at Work" is defined and its implications for insurance eligibility.'